In [ ]:
# pass

# Test access to various data sources

In [ ]:
import os
import numpy as np
import pandas as pd
import requests
from lsst_efd_client import EfdClient

try:
    from lsst.summit.utils import ConsDbClient
    has_summit_utils = True
except ImportError:
    # No lsst.summit.utils
    has_summit_utils = False

# Note that access tokens are RSP platform specific
try:
    from lsst.rsp import get_access_token
except ImportError:
    def get_access_token(token_file : str | None = None) -> str:
        token = os.environ.get("ACCESS_TOKEN")
        if token == None:
            if token_file is not None:
                with open("token_file", "r") as f:
                    token = f.read()
            else:
                warnings.warn("No RSP token available.")
        return token


def get_clients() -> dict:
    """Return site-specific client connections. 

    Returns
    -------
    endpoints : `dict`
        Dictionary with `efd`, `obsenv`, 
        `narrative_log`, and `exposure_log`
        connection information.
        For the obsenv, narrative log and exposure log, these are only
        defined for the summit or USDF.
    """
    # Set up authentication
    token = get_access_token()
    auth = ("user", token)
    # This authentication is for nightlog, exposurelog, nightreport currently
    # But I think it's the same underlying info for EfdClient i.e.
    # https://github.com/lsst/schedview/blob/e11fbd51ee5e22d11fef9a52f66dfcc082181cb6/schedview/app/scheduler_dashboard/influxdb_client.py
    
    # let's do this like lsst.summit.utils.getSite but simpler
    site = "UNKNOWN"
    location = os.getenv("EXTERNAL_INSTANCE_URL", "")
    if "tucson-teststand" in location:
        site = "tucson"
    elif "summit-lsp" in location:
        site = "summit"
    elif "base-lsp" in location:
        site = "base"
    elif "usdf-rsp" in location:
        site = "usdf"
    # If location not set, next step is to check hostname
    elif location == "":
        hostname = os.getenv("HOSTNAME", "")
        interactiveNodes = ("sdfrome", "sdfiana")
        if hostname.startswith(interactiveNodes):
            site = "usdf"
        elif hostname == "htcondor.ls.lsst.org":
            site = "base"
        elif hostname == "htcondor.cp.lsst.org":
            site = "summit"
    # If none of the above, use usdf again.
    if site == "UNKNOWN":
        site = "usdf"
    
    if site == "summit":
        api_base = "https://summit-lsp.lsst.codes"
        efd_client = EfdClient("summit_efd")
        obsenv_client = EfdClient("summit_efd", db_name="lsst.obsenv")
    elif site == "tucson":
        api_base = None
        efd_client = EfdClient("tucson_teststand_efd")
        obsenv_client = EfdClient("tucson_teststand_efd", db_name="lsst.obsenv")
    elif site == "base":
        api_base = "https://base-lsp.slac.lsst.codes"
        efd_client = EfdClient("base_efd")
        obsenv_client = EfdClient("base_efd", db_name="lsst.obsenv")
    elif site == "usdf":
        # For tokens, need to distinguish between dev and prod
        if "dev" in location:
            api_base = "https://usdf-rsp-dev.slac.stanford.edu"
        else:
            api_base = "https://usdf-rsp.slac.stanford.edu"
        efd_client = EfdClient("usdf_efd")
        obsenv_client = EfdClient("usdf_efd", db_name='lsst.obsenv')
    else:
        # Assume USDF prod
        efd_client = EfdClient("usdf_efd")
        obsenv_client = EfdClient("usdf_efd", db_name='lsst.obsenv')
        api_base = "https://usdf-rsp.slac.stanford.edu"
    narrative_log =  "/narrativelog/messages"
    exposure_log = "/exposurelog/messages"
    nightreport =  "/nightreport/reports"

    endpoints = {'api_base': api_base, 'auth': auth, 
                'efd': efd_client, 'obsenv': obsenv_client, 
                'narrative_log': narrative_log, 'exposure_log': exposure_log,  
                'nightreport': nightreport}
    
    # If some verbose output is desired
    # We'll put this here to make it easier to avoid printing the auth token
    endpoints_string = f"base url: {endpoints['api_base']} " 
    endpoints_string += f"efd host: {endpoints['efd'].influx_client.host}"
    endpoints['string'] = endpoints_string

    return endpoints

def query_logging_services(endpoint: str, auth: dict, params: dict) -> pd.DataFrame:
    """Send query to narrative log or exposure log services.
    
    Parameters
    ----------
    endpoint : `str`
        The URL to send the query to.
        Usually like `https://usdf-rsp.slac.stanford.edu/narrativelog/messages`
    auth : `dict`
        The username and password for authentication.
        The username can be any string, the password should be an RSP token.
        See e.g. https://nb.lsst.io/environment/tokens.html 
    params : `dict`
        Dictionary of parameters for the REST API query.
        See docs for each service for more details

    Returns
    -------
    messages : `pd.DataFrame`
        The returned log messages (if any available), in a dataframe.
    """
    # Very often, requests from the logging endpoints fail the first time.
    response = requests.get(endpoint, auth=auth, params=params)
    # Try twice.
    if response.status_code != 200:
        response = requests.get(endpoint, auth=auth, params=params)
    if response.status_code != 200:
        err_string = f"{endpoint} "
        err_string += " unavailable."
        print(err_string)
        print(response)
        print(response.status_code)
        messages = []
    else:
        messages = response.json()
    messages = pd.DataFrame(messages)
    return messages

In [ ]:
endpoints = get_clients()
print(endpoints['string'])

In [ ]:
print("is there an rsp token of non-zero length?")
if endpoints['auth'] is None:
    print("Auth token retrieval failed")
print("Auth token has a length ", len(endpoints['auth']))

In [ ]:
from IPython.display import display, HTML

log_limit = 50000
params = {"is_human" : "either",
          "is_valid" : "true",
          "has_date_begin" : True,
          #"min_date_begin" : t_start.to_datetime(),
          #"max_date_begin" : t_end.to_datetime(),
          "order_by" : "date_begin",
          "limit": log_limit, 
         }
messages = query_logging_services(endpoints['api_base'] + endpoints['narrative_log'], auth=endpoints['auth'], params=params)

display(HTML(messages.head().to_html()))

In [ ]:
log_limit = 50000
#min_dayobs_int = day_obs_min.replace("-", '')
#max_dayobs_int = day_obs_max.replace("-", '')
params = {"is_human" : "either",
          "is_valid" : "true",
          #"min_day_obs" : min_dayobs_int,
          #"max_day_obs" : max_dayobs_int,
          "limit": log_limit, 
         }
messages = query_logging_services(endpoints['api_base'] + endpoints['exposure_log'], auth=endpoints['auth'], params=params)

display(HTML(messages.head().to_html()))

In [ ]:
params = {#"telescopes" : tel_nr,
          #"min_day_obs" : this_dayobs,
          #"max_day_obs" : next_dayobs,
          "is_valid" : "true",
         }

nightlog = query_logging_services(endpoints['api_base'] + endpoints['nightreport'], auth=endpoints['auth'], params=params)

display(HTML(nightlog.head().to_html()))